In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)
type(train_dataset)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


numpy.ndarray

In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
# Adding regularization to the 1 hidden layer network
graph1 = tf.Graph()
batch_size = 128

num_steps=3001

import datetime
startTime = datetime.datetime.now() 

def define_and_run_batch(beta):
    
    num_RELU =1024
    
    with graph1.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)
  
      # Variables.
      weights_RELU = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_RELU]))
      biases_RELU = tf.Variable(tf.zeros([num_RELU]))
      weights_layer1 = tf.Variable(
        tf.truncated_normal([num_RELU, num_labels]))
      biases_layer1 = tf.Variable(tf.zeros([num_labels]))
  
      # Training computation.
      logits_RELU = tf.matmul(tf_train_dataset, weights_RELU) + biases_RELU
      RELU_vec = tf.nn.relu(logits_RELU)
      logits_layer = tf.matmul(RELU_vec, weights_layer1) + biases_layer1                  
      # loss = tf.reduce_mean(
      #        tf.nn.softmax_cross_entropy_with_logits(logits_layer, tf_train_labels))
      cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits_layer, tf_train_labels,name="cross_entropy")
      l2reg = tf.reduce_sum(tf.square(weights_RELU))+tf.reduce_sum(tf.square(weights_layer1))
      # beta = 0.005
      loss = tf.reduce_mean(cross_entropy+beta*l2reg)
  
  # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
  
      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits_layer)
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu((tf.matmul(tf_valid_dataset, weights_RELU) + biases_RELU)),weights_layer1)+biases_layer1)
                         
      test_prediction =tf.nn.softmax(
        tf.matmul(tf.nn.relu((tf.matmul(tf_test_dataset, weights_RELU) + biases_RELU)),weights_layer1)+biases_layer1)
                         
    with tf.Session(graph=graph1) as session:
 
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch. 
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        #
        
        _, l, predictions, logits = session.run(
          [optimizer, loss,train_prediction,logits_RELU], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      test_acc = accuracy(test_prediction.eval(), test_labels)
      print("Test accuracy: %.1f%%" % test_acc)

      print('loss=%s' % l)
    x = datetime.datetime.now() - startTime
    print(x)
    return(test_acc,round(l,5))
    
define_and_run_batch(0.005)

Initialized
Minibatch loss at step 0: 3485.660889
Minibatch accuracy: 4.7%
Validation accuracy: 24.6%
Minibatch loss at step 500: 154.070740
Minibatch accuracy: 87.5%
Validation accuracy: 82.9%
Minibatch loss at step 1000: 8.342560
Minibatch accuracy: 84.4%
Validation accuracy: 84.4%
Minibatch loss at step 1500: 0.996325
Minibatch accuracy: 88.3%
Validation accuracy: 84.1%
Minibatch loss at step 2000: 0.638780
Minibatch accuracy: 89.8%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 0.649237
Minibatch accuracy: 83.6%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 0.762351
Minibatch accuracy: 82.8%
Validation accuracy: 84.0%
Test accuracy: 90.6%
loss=0.762351
0:00:55.490985


(90.59, 0.76235)

In [7]:
beta_list = np.array([0,0.00001,0.000033,0.0001,0.00033,0.001,0.0033,0.01,0.033,0.1,0.33,1,3.33,10])

In [9]:
max_range=beta_list.shape[0]
res = np.empty([max_range,2])

for i in range(0,max_range):
    print('beta = %s' % beta_list[i])    
    acc,l = define_and_run_batch(beta_list[i])
    res [i,0]= acc
    res [i,1]= l

beta = 0.0
Initialized
Minibatch loss at step 0: 345.184143
Minibatch accuracy: 10.9%
Validation accuracy: 26.2%
Minibatch loss at step 500: 16.914642
Minibatch accuracy: 83.6%
Validation accuracy: 82.0%
Minibatch loss at step 1000: 21.335110
Minibatch accuracy: 74.2%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 6.546029
Minibatch accuracy: 88.3%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 8.219948
Minibatch accuracy: 82.0%
Validation accuracy: 83.6%
Minibatch loss at step 2500: 2.744931
Minibatch accuracy: 89.1%
Validation accuracy: 83.4%
Minibatch loss at step 3000: 4.664391
Minibatch accuracy: 77.3%
Validation accuracy: 83.1%
Test accuracy: 90.0%
loss=4.66439
0:10:57.858656
beta = 1e-05
Initialized
Minibatch loss at step 0: 411.308472
Minibatch accuracy: 8.6%
Validation accuracy: 30.4%
Minibatch loss at step 500: 23.506004
Minibatch accuracy: 84.4%
Validation accuracy: 81.9%
Minibatch loss at step 1000: 14.518665
Minibatch accuracy: 82.8%
Validation accuracy: 

In [10]:
res

array([[ 89.97   ,   4.66439],
       [ 89.9    ,   8.69076],
       [ 90.3    ,  21.35772],
       [ 90.14   ,  44.55725],
       [ 91.39   ,  61.67263],
       [ 93.48   ,  17.17248],
       [ 91.39   ,   0.71721],
       [ 89.28   ,   0.88581],
       [ 87.36   ,   1.2519 ],
       [ 80.43   ,   1.79164],
       [ 23.16   ,   2.32489],
       [ 10.     ,   2.30044],
       [ 10.     ,       nan],
       [ 10.     ,       nan]])

In [55]:
round(88.2423,2)

88.24